## Imports

In [1]:
import pandas as pd
import numpy as np

In [2]:
from unidecode import unidecode
import re
import os
from countryinfo import CountryInfo
from googletrans import Translator

## Fuentes

A continuación se listan las url donde se obtienen los datos

 * Medallas ganadas por país

In [3]:
url_medallas = "https://en.wikipedia.org/wiki/All-time_Olympic_Games_medal_table"

 * Lista de países por continente

In [4]:
url_paises = "https://es.wikipedia.org/wiki/Anexo:Pa%C3%ADses_por_continentes"

 * Equivalencias entre códigos DOI y ISO

In [5]:
url_codigos = "https://es.wikipedia.org/wiki/Anexo:Comparaci%C3%B3n_de_los_c%C3%B3digos_del_COI,_la_FIFA_y_la_ISO_3166"

Los dataframes generados se guardarán en el directorio llamado `data/`, bajo los siguientes nombres,
 * `info_countries.csv`: Información variada de los países
 * `olympics.csv`: Información de medallas ganadas por país
 * `olympics_excel.xlsx`: Información de medallas ganadas por país en formato excel

In [6]:
data_path = "data"
if not os.path.exists(data_path):
	os.mkdir(data_path)

## Funciones a utilizar

La siguiente función se usará para darle un formato más cómodo a las columnas del dataframe resultante

In [7]:
def tratamiento_columnas(dataframe):

	columns = dataframe.columns

	new_columns = []
	for col in columns:

		if not isinstance(col, str):
			new_columns.append(col)
			continue

		col = unidecode(col.lower()).strip()
		while "  " in col:
			col = col.replace("  ", " ")
		col = re.sub(r"\s", "_", col)

		if "superficie" in col:
			col = "superficie_km2"
		elif "poblacion" in col:
			col = "poblacion"
		elif "iso" in col:
			col = col.replace("iso", "ISO")
		elif "idh" in col:
			col = col.upper()
		elif col in ["nombre_oficial", "pais", "dependencia", "region_ultraperiferica", "region"]:
			col = "nombre"

		new_columns.append(col)

	dataframe.columns = new_columns

	return dataframe

## Extracción de los dataframes

#### Dataframe de medallas por país

A continuación se extrae la información que Wikipedia posee de las medallas olímpicas ganadas por país, tanto en olimpiadas de verano como en las de invierno.

Primero extraemos de la url correspondiente.

In [8]:
df = pd.read_html(url_medallas)[1]

Renombramos las columnas para que la manipulación del data frame sea más cómodo

In [9]:
columns = ['Country', 'Num_Summer', 'gold_summer', 'silver_summer',
       'bronze_summer', 'Total_summer', 'Num_Winter', 'gold_winter',
       'silver_winter', 'bronze_winter', 'Total_winter', 'Num_Games', 'gold',
       'silver', 'bronze', 'Combined_total']

df.columns = columns
df = df[df["Country"] != "Totals"]

Una columna posee tanto el nombre del país como el código COI, por lo que crearemos dos columnas más donde los separaremos

In [10]:
df["Country_code_coi"] = df.Country.str.extract(r"\((\w{3})\)")[0]
df["Country_name"] = df.Country.str.extract(r"(.*)\s\(")[0]
df["Country"] = df["Country_name"] + " (" + df["Country_code_coi"] + ")"
df = df[['Country', 'Country_name','Country_code_coi', 'Num_Summer', 'gold_summer', 'silver_summer',
       'bronze_summer', 'Total_summer', 'Num_Winter', 'gold_winter',
       'silver_winter', 'bronze_winter', 'Total_winter', 'Num_Games', 'gold',
       'silver', 'bronze', 'Combined_total']]

Las columnas del dataframe están listas y son las siguientes
 * `Country`: Nombre del país y código expedido por el Comité Olímpico Internacional.
 * `Country_name`: Nombre del país.
 * `Country_code_coi`: Código expedido por el Comité Olímpico Internacional.
 * `Num_Summer`: Número de participaciones en Olimpiadas de Verano.
 * `gold_summer`: Número de medallas de oro ganadas en Olimpiadas de Verano.
 * `silver_summer`: Número de medallas de plata ganadas en Olimpiadas de Verano.
 * `bronze_summer`: Número de medallas de bronce ganadas en Olimpiadas de Verano.
 * `Total_summer`: Total de medallas de ganadas en Olimpiadas de Verano.
 * `Num_Winter`: Número de participaciones en Olimpiadas de Invierno.
 * `gold_winter`: Número de medallas de oro ganadas en Olimpiadas de Invierno.
 * `silver_winter`: Número de medallas de plata ganadas en Olimpiadas de Invierno.
 * `bronze_winter`: Número de medallas de bronce ganadas en Olimpiadas de Invierno.
 * `Total_winter`: Total de medallas de ganadas en Olimpiadas de Invierno.
 * `Num_Games`: Total de participaciones.
 * `gold`: Total de medallas de oro ganadas.
 * `silver`: Total de medallas de plata ganadas.
 * `bronze`: Total de medallas de bronce ganadas.
 * `Combined_total`: Total de medallas combinadas.

El dataframe queda de la siguiente manera 

In [11]:
df.sample(10)

,Country,Country_name,Country_code_coi,Num_Summer,gold_summer,silver_summer,bronze_summer,Total_summer,Num_Winter,gold_winter,silver_winter,bronze_winter,Total_winter,Num_Games,gold,silver,bronze,Combined_total
153,Vietnam (VIE),Vietnam,VIE,17,1,3,1,5,0,0,0,0,0,17,1,3,1,5
72,Jordan (JOR),Jordan,JOR,12,1,2,1,4,0,0,0,0,0,12,1,2,1,4
137,Tajikistan (TJK),Tajikistan,TJK,8,1,1,5,7,4,0,0,0,0,12,1,1,5,7
112,Russia (RUS),Russia,RUS,6,147,126,150,423,6,47,39,35,121,12,194,165,185,544
22,Cameroon (CMR),Cameroon,CMR,16,3,1,2,6,1,0,0,0,0,17,3,1,2,6
68,Israel (ISR),Israel,ISR,18,4,6,10,20,8,0,0,0,0,26,4,6,10,20
110,Refugee Olympic Team (EOR),Refugee Olympic Team,EOR,3,0,0,1,1,0,0,0,0,0,3,0,0,1,1
14,Bermuda (BER),Bermuda,BER,20,1,0,1,2,8,0,0,0,0,28,1,0,1,2
1,Albania (ALB),Albania,ALB,10,0,0,2,2,5,0,0,0,0,15,0,0,2,2
11,Barbados (BAR),Barbados,BAR,14,0,0,1,1,0,0,0,0,0,14,0,0,1,1


In [12]:
df.shape

(162, 18)

#### Dataframe de paises

Con la finalidad de conocer un poco mejor a los países, vamos a importar un dataframe con un poco de su información.

Primero vamos con información relevante. Primer crearemos una función que permitirá hacer la extracción y el tratamiento correspondiente:

In [13]:
def validar_continente(pais, series):
	return any(map(lambda x: bool(re.search(pais, x)), series.values))

def titulos_tratamiento(string):
	pattern = re.compile(r"(.*?)(?:\[.+\])+.*(\(.+\))?")

	if (match_ := pattern.fullmatch(string)):
		return match_.group(1)
	else:
		return string

def data_frame_paises():
	mapping = {
		"Alemania" : "Europa",
		"Kosovo" : "Europa", # Estados con reconocimiento limitado
		"China" : "Asia",
		"Israel" : "Asia",  # Estados con reconocimiento limitado
		"Isla de Navidad" : "Asia", # Territorios dependientes
		"Macao": "Asia", # Regiones administrativas especiales de la República Popular de China
		"Angola" : "Africa",
		"Chile" : "América",
		"Anguila" : "América", # Territorios dependientes
		"Guadalupe" : "América", # Países americanos integrados en estados no americanos
		"Australia" : "Oceanía",
		"Guam" : "Oceanía" # Territorios dependientes
	}

	df_continents = pd.read_html(url_paises)

	tablas = []

	df_tratados = map(tratamiento_columnas, df_continents)

	for tb in df_tratados:
		try:
			map_country = list(filter(lambda x: validar_continente(x, tb["nombre"]), mapping.keys()))[0]
		except (KeyError, IndexError) as e:
			#print(e)
			continue

		tb["Continente"] = mapping[map_country]
		tablas.append(tb.copy())

	df_countries = pd.concat(tablas, axis=0)
	df_countries["nombre"] = df_countries["nombre"].map(titulos_tratamiento)

	return df_countries

def extraer_codigo_iso(series):
	traducidos = {
		'Botsuana': 'Botswana',
		'Comoras': 'Comoros',
		'Gambia': 'Republic of The Gambia',
		'Guinea-Bisáu': 'Guinea-Bissau',
		'Malaui': 'Malawi',
		'Mauricio': 'Mauritius',
		'República Democrática del Congo': 'Democratic Republic of the Congo',
		'República del Congo': 'Republic of the Congo',
		'Santo Tomé y Príncipe': 'São Tomé and Príncipe',
		'Somalilandia (República de Somalilandia)': 'Somaliland',
		'Suazilandia': 'Swaziland',  # Suazilandia cambió su nombre a Eswatini en 2018
		'Sudán del Sur (República de Sudán del Sur)': 'South Sudan',
		'Yibuti': 'Djibouti'
	}

	if isinstance(series["ISO_3166-1"], str):
		if len(series["ISO_3166-1"]) == 3:
			return series["ISO_3166-1"]

	t = Translator()
	pais_ = series.nombre

	pais_ingles = t.translate(pais_, src="es", dest="en")
	countryinfo = CountryInfo(pais_ingles.text)

	try:
		return countryinfo.iso(3)
	except KeyError:
		try:
			countryinfo = CountryInfo(traducidos[pais_])
			return countryinfo.iso(3)
		except KeyError:
			#print(f"No se pudo obtener código ISO de {pais_}")
			return np.nan

In [14]:
df_countries = data_frame_paises()
df_countries = df_countries.dropna(axis=1, how="all")
df_countries = df_countries.rename(columns={"codigo_ISO_3166-1" : "ISO_3166-1"})
df_countries["ISO_3166-1"] = df_countries.apply(extraer_codigo_iso, axis=1)
df_countries[df_countries["ISO_3166-1"].isna()].to_csv(os.path.join(data_path, "info_countries_ISO_nan.csv"), index=False, encoding="utf-8")
df_countries = df_countries.dropna(subset="ISO_3166-1")
df_countries.sample(10)

,nombre,ISO_3166-1,superficie_km2,poblacion,capital,idiomas_oficiales,moneda,Continente,estatus,idioma_oficial,pib_per_capita_(us$),IDH,tipo_de_dependencia,idiomas,territorio
21,Lesoto,LSO,30 355,1 795 000,Maseru,NaN,Loti,Africa,NaN,SesotoInglés,$2113,NaN,NaN,NaN,NaN
23,Islandia,ISL,103.000,335.878,Reikiavik,Islandés,"Corona islandesa(kr., ISK).",Europa,NaN,NaN,NaN,NaN,NaN,NaN,NaN
43,Somalia,SOM,637 661,17 700 000,Mogadiscio,NaN,Chelín somalí,Africa,NaN,SomalíÁrabe,$2941,NaN,NaN,NaN,NaN
52,Uganda,UGA,236 040,27 616 000,Kampala,NaN,Chelín ugandés,Africa,NaN,InglésSuajili,$1700,NaN,NaN,NaN,NaN
16,Wallis y Futuna,WLF,274,16 000,Mata Utu,NaN,NaN,Asia,NaN,NaN,NaN,NaN,Francia,Francés y Walisiano,Colectividad Francesa de Ultramar
32,Níger,NER,1 267 000,13 957 000,Niamey,NaN,Franco CFA de África Occidental,Africa,NaN,Francés,$700,NaN,NaN,NaN,NaN
36,República del Congo,COG,342 000,3 999 000,Brazzaville,NaN,Franco CFA de África Central,Africa,NaN,FrancésLingala,$1369,NaN,NaN,NaN,NaN
11,Cuba,CUB,110 860,11 179 995,La Habana,Español,"Peso cubano($, CUP),Peso cubano convertible($,...",América,NaN,NaN,NaN,NaN,NaN,NaN,NaN
16,Ghana,GHA,238 534,23 000 000,Acra,NaN,Cedi,Africa,NaN,Inglés,$2700,NaN,NaN,NaN,NaN
6,Islas Caimán,CYM,264,57 268,George Town,Inglés,"Dólar de las Islas Caimán($, KYD)",América,Territorio Británico de Ultramar,NaN,NaN,NaN,NaN,NaN,NaN


Dado que el código ISO y el código dado por el COI en ocasiones no coinciden, es necesario tener la comparación entre las dos, para eso usaremos otro dataframe

In [15]:
df_codigos = pd.read_html(url_codigos)[1]
df_codigos.columns = [
	"nombre", "COI", "FIFA", "ISO", "Observaciones"
]
df_codigos["ISO"] = df_codigos["ISO"].dropna()
df_codigos

,nombre,COI,FIFA,ISO,Observaciones
0,Afganistán,AFG,AFG,AFG,NaN
1,Åland,NaN,NaN,ALA,No es un país. Pertenece a Finlandia.
2,Albania,ALB,ALB,ALB,NaN
3,Alemania,GER,GER,DEU,NaN
4,Andorra,AND,AND,AND,NaN
...,...,...,...,...,...
249,Yemen,YEM,YEM,YEM,NaN
250,Yibuti,DJI,DJI,DJI,NaN
251,Wallis y Futuna,NaN,NaN,WLF,No es un país. Pertenece a Francia.
252,Zambia,ZAM,ZAM,ZMB,NaN


Solo queda unir ambos dataframes

In [16]:
df_countries.shape

(229, 15)

In [17]:
df_countries = pd.merge(df_countries, df_codigos[["COI", "FIFA", "ISO"]], how="left", left_on="ISO_3166-1", right_on="ISO").drop(columns=["ISO"])
df_countries = df_countries[['nombre', 'ISO_3166-1', "COI", "FIFA", 'superficie_km2', 'poblacion', 'capital','idiomas_oficiales', 'moneda', 'Continente']]

In [18]:
df_countries.shape

(229, 10)

El dataframe de paises queda con las siguientes columnas
 * `pais`: Nombre de país en español.
 * `ISO_3166-1`: Código ISO 3166-1 alpha 3 dado al país.
 * `ISO`: 
 * `superficie_km2`: Extensión territorial dado en $m^2$.
 * `poblacion`: Número de habitantes.
 * `capital`: Capital administrativa del país.
 * `idiomas_oficiales`: Idiomas oficiales.
 * `moneda`: Moneda de curso legal.
 * `Continente`: Continente al que pertenecen: América, Asia, Africa, Europa, Oceania.
 * `IDH`: Índice de Desarrollo Humano.

In [19]:
df_countries.to_csv(os.path.join(data_path, "info_countries.csv"), index=False, encoding="utf-8")
df_countries.sample(10)

,nombre,ISO_3166-1,COI,FIFA,superficie_km2,poblacion,capital,idiomas_oficiales,moneda,Continente
177,Ecuador,ECU,ECU,ECU,283 561,16 080 778,Quito,Español,"Dólar estadounidense($, USD).",América
129,Kenia,KEN,KEN,KEN,580 367,34 707 817,Nairobi,NaN,Chelín keniano,Africa
29,Luxemburgo,LUX,LUX,LUX,2.586,582.291,Ciudad de Luxemburgo,LuxemburguésFrancésAlemán,"Euro(€, EUR).",Europa
68,Emiratos Árabes Unidos,ARE,UAE,UAE,83 600,5 927 482,Abu Dabi,Árabe,"Dírham de los Emiratos Árabes Unidos(Dhs, AED).",Asia
77,Kazajistán,KAZ,KAZ,KAZ,2 724 900,18 360 353,Astaná,KazajoRuso,"Tenge kazajo(T, KZT).",Asia
178,El Salvador,SLV,ESA,SLV,21 041,6 156 670,San Salvador,Español,"Dólar estadounidense($, USD).",América
117,Chad,TCD,CHA,CHA,1 284 000,10 146 000,Yamena,NaN,Franco CFA de África Central,Africa
145,República del Congo,COG,CGO,CGO,342 000,3 999 000,Brazzaville,NaN,Franco CFA de África Central,Africa
71,India,IND,IND,IND,3 287 263[10]​,1 266 883 598,Nueva Delhi,HindiInglés,"Rupia india(₹, INR).",Asia
219,Tuvalu,TUV,TUV,NaN,26,11 000,Funafuti,Tuvaluano e Inglés,Dólar tuvaluano y dólar australiano,Oceanía


#### Creando dataframe de trabajo

Ahora vamos a complementar el dataframe de medallas con algunos datos de los países. Por supuesto, en las Olimpiadas no solo participan países, sino también algunas organizaciones, sin embargo, no los tomaremos en cuenta.

Comencemos con algo simple: agregando el código ISO y el continente al df de medallas

In [20]:
df = df.merge(df_countries[["ISO_3166-1", "COI", "Continente"]], how="left", left_on="Country_code_coi", right_on="COI").drop(columns=["COI"])
df = df[['Country', 'Country_name','Country_code_coi', "ISO_3166-1", "Continente", 'Num_Summer', 'gold_summer', 'silver_summer',
       'bronze_summer', 'Total_summer', 'Num_Winter', 'gold_winter',
       'silver_winter', 'bronze_winter', 'Total_winter', 'Num_Games', 'gold',
       'silver', 'bronze', 'Combined_total']]
df = df.drop_duplicates()
df.sample(5)

,Country,Country_name,Country_code_coi,ISO_3166-1,Continente,Num_Summer,gold_summer,silver_summer,bronze_summer,Total_summer,Num_Winter,gold_winter,silver_winter,bronze_winter,Total_winter,Num_Games,gold,silver,bronze,Combined_total
55,United Team of Germany (EUA),United Team of Germany,EUA,NaN,NaN,3,28,54,36,118,3,8,6,5,19,6,36,60,41,137
148,Togo (TOG),Togo,TOG,TGO,Africa,12,0,0,1,1,2,0,0,0,0,14,0,0,1,1
97,Morocco (MAR),Morocco,MAR,MAR,Africa,16,8,5,13,26,8,0,0,0,0,24,8,5,13,26
103,Niger (NIG),Niger,NIG,NER,Africa,14,0,1,1,2,0,0,0,0,0,14,0,1,1,2
85,Kyrgyzstan (KGZ),Kyrgyzstan,KGZ,KGZ,Asia,8,0,5,8,13,8,0,0,0,0,16,0,5,8,13


In [21]:
df.shape

(170, 20)

Posteriormente, es importante tratar aquellos registros que no pudieron ser clasificados por continentes, o bien, que no obtuvieron un código ISO. Como los siguientes ejemplos,

In [25]:
df[df.Continente.isna() | df["ISO_3166-1"].isna()].sample(5)

,Country,Country_name,Country_code_coi,ISO_3166-1,Continente,Num_Summer,gold_summer,silver_summer,bronze_summer,Total_summer,Num_Winter,gold_winter,silver_winter,bronze_winter,Total_winter,Num_Games,gold,silver,bronze,Combined_total
6,Australasia (ANZ),Australasia,ANZ,NaN,NaN,2,3,4,5,12,0,0,0,0,0,2,3,4,5,12
57,West Germany (FRG),West Germany,FRG,NaN,NaN,5,56,67,81,204,6,11,15,13,39,11,67,82,94,243
169,Independent Olympic Participants (IOP),Independent Olympic Participants,IOP,NaN,NaN,1,0,1,2,3,0,0,0,0,0,1,0,1,2,3
124,ROC (ROC),ROC,ROC,NaN,NaN,1,20,28,23,71,1,5,12,15,32,2,25,40,38,103
154,Turkmenistan (TKM),Turkmenistan,TKM,NaN,NaN,8,0,1,0,1,0,0,0,0,0,8,0,1,0,1


In [27]:
df[df.Continente.isna() | df["ISO_3166-1"].isna()]["Country_name"].tolist()

['Australasia',
 'Bohemia',
 'British West Indies',
 'Czechoslovakia',
 'United Team of Germany',
 'East Germany',
 'West Germany',
 'Kosovo',
 'Netherlands Antilles',
 'Refugee Olympic Team',
 'Russian Empire',
 'Soviet Union',
 'Unified Team',
 'Olympic Athletes from Russia',
 'ROC',
 'Serbia and Montenegro',
 'Turkmenistan',
 'Virgin Islands',
 'Yugoslavia',
 'Individual Neutral Athletes',
 'Independent Olympic Athletes',
 'Independent Olympic Participants',
 'Mixed team']

In [23]:
df[df.duplicated(subset=["Country_code_coi"], keep=False)]

,Country,Country_name,Country_code_coi,ISO_3166-1,Continente,Num_Summer,gold_summer,silver_summer,bronze_summer,Total_summer,Num_Winter,gold_winter,silver_winter,bronze_winter,Total_winter,Num_Games,gold,silver,bronze,Combined_total
4,Armenia (ARM),Armenia,ARM,ARM,Europa,8,2,11,9,22,8,0,0,0,0,16,2,11,9,22
5,Armenia (ARM),Armenia,ARM,ARM,Asia,8,2,11,9,22,8,0,0,0,0,16,2,11,9,22
9,Azerbaijan (AZE),Azerbaijan,AZE,AZE,Europa,8,9,16,31,56,7,0,0,0,0,15,9,16,31,56
10,Azerbaijan (AZE),Azerbaijan,AZE,AZE,Asia,8,9,16,31,56,7,0,0,0,0,15,9,16,31,56
34,Cyprus (CYP),Cyprus,CYP,CYP,Europa,12,0,2,0,2,12,0,0,0,0,24,0,2,0,2
35,Cyprus (CYP),Cyprus,CYP,CYP,Asia,12,0,2,0,2,12,0,0,0,0,24,0,2,0,2
43,Egypt (EGY),Egypt,EGY,EGY,Asia,24,9,12,20,41,1,0,0,0,0,25,9,12,20,41
44,Egypt (EGY),Egypt,EGY,EGY,Africa,24,9,12,20,41,1,0,0,0,0,25,9,12,20,41
52,Georgia (GEO),Georgia,GEO,GEO,Europa,8,13,15,19,47,8,0,0,0,0,16,13,15,19,47
53,Georgia (GEO),Georgia,GEO,GEO,Asia,8,13,15,19,47,8,0,0,0,0,16,13,15,19,47


In [24]:
df.to_csv(os.path.join(data_path,"olympics.csv"), index=False, encoding="utf-8")
df.to_excel(os.path.join(data_path,"olympics_excel.xlsx"), index=False, encoding="utf-8")